In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import mmread
from sklearn.preprocessing import normalize
import anndata as ad
import matplotlib as mpl
from harmony import harmonize
sc.set_figure_params(figsize=(6, 6), frameon=False)
sc.settings.n_jobs=8

In [ ]:
HCC_1T = './raw_data/Primary_Tumor/HCC-1T'
HCC_1L = './raw_data/Leading_Edge/HCC-1L'
HCC_1N = './raw_data/Adjacent/HCC-1N'

HCC_2T = './raw_data/Primary_Tumor/HCC-2T'
HCC_2L = './raw_data/Leading_Edge/HCC-2L'
HCC_2N = './raw_data/Adjacent/HCC-2N'

HCC_3T = './raw_data/Primary_Tumor/HCC-3T'
HCC_3L = './raw_data/Leading_Edge/HCC-3L'
HCC_3N = './raw_data/Adjacent/HCC-3N'

HCC_4T = './raw_data/Primary_Tumor/HCC-4T'
HCC_4L = './raw_data/Leading_Edge/HCC-4L'
HCC_4N = './raw_data/Adjacent/HCC-4N'

visium_paths = [HCC_1T, HCC_1L, HCC_1N, HCC_2T, HCC_2L, HCC_2N,
                HCC_3T, HCC_3L, HCC_3N, HCC_4T, HCC_4L, HCC_4N]
tissue_names_unique = ['Patient 1 Primary Tumor', 
                       'Patient 1 Leading Edge', 
                       'Patient 1 Ajacent Area',
                       'Patient 2 Primary Tumor', 
                       'Patient 2 Leading Edge', 
                       'Patient 2 Ajacent Area', 
                       'Patient 3 Primary Tumor', 
                       'Patient 3 Leading Edge', 
                       'Patient 3 Ajacent Area',
                       'Patient 4 Primary Tumor', 
                       'Patient 4 Leading Edge', 
                       'Patient 4 Ajacent Area']

In [ ]:
adata_l = []
for i in range(12):
    adata = sc.read_visium(visium_paths[i])
    adata_l.append(adata)

In [ ]:
def plot_globally(gene, title, dpi = 20):
    fig, ax = plt.subplots(4,3, figsize=(35, 50))
    sc.set_figure_params(fontsize = 50, dpi=dpi, dpi_save=dpi)
    dark_image_idx = [3, 4, 9, 10, 11]
    for p in range(4):
        for t in range(3):

            idx = 3*p + t

            if idx in dark_image_idx:
                alpha_img = 0.6
                alpha = 0.7
            else:
                alpha_img = 0.3
                alpha = 0.9

            adata = adata_l[idx]
            sc.pl.spatial(adata, img_key="hires", color=gene, 
                          alpha = alpha, alpha_img = alpha_img, ax = ax[p, t], 
                          title = title, vmin = 0, 
                          color_map = 'Greens', colorbar_loc = 'right', show = False)
    plt.show()    

# Read in Ligand-receptor data

In [ ]:
import rdata

In [ ]:
parsed = rdata.parser.parse_file("./Ligand-Receptor/CellChatDB.human.rda")
converted = rdata.conversion.convert(parsed)

In [ ]:
#filter out all overlapping genes
LR = converted['CellChatDB.human']['interaction'].loc[:, ['ligand', 'receptor']]
ST_genes = list(adata_l[0].var_names)
LR_common_genes = []
for i in range(LR.shape[0]):
    if (LR.iloc[i,0] in ST_genes) and (LR.iloc[i,1] in ST_genes):
        LR_common_genes.append((LR.iloc[i,0], LR.iloc[i,1]))

In [ ]:
len(LR_common_genes)

# Lujia's LR Data

In [ ]:
LR_pairs = pd.read_csv("./Ligand-Receptor/Lujia_lr_pairs.csv", header = 0)
LR_common_genes = []
for i in range(LR_pairs.shape[0]):
    if (LR_pairs.iloc[i,0] in ST_genes) and (LR_pairs.iloc[i,1] in ST_genes):
        LR_common_genes.append((LR_pairs.iloc[i,0], LR_pairs.iloc[i,1]))

In [ ]:
len(LR_common_genes)

# Screening Ligand Receptor 

In [ ]:
#if slide express more than 5%(threshold) ligand-rectptor pair, keep, else, discard this slide
threshold = 0.05
keeped_pairs = {}
for i in range(len(LR_common_genes)):
    if i % 200 == 0:
        print('Progress: ', i)
    ligand = LR_common_genes[i][0]
    receptor = LR_common_genes[i][1]
    
    kept_slide_idx = []
    for j in range(12):
        adata = adata_l[j]
        adata.var_names_make_unique()
        ligand_exp = adata[:, ligand].X.toarray().flatten()
        receptor_exp = adata[:, receptor].X.toarray().flatten()
        product = ligand_exp * receptor_exp
        
        if (np.count_nonzero(product) > int(threshold * len(product))):
            kept_slide_idx.append(j)
            
    keeped_pairs[LR_common_genes[i]] = kept_slide_idx
        
print("%d pairs were kept!" %len(keeped_pairs))
    

In [ ]:
#define ligand-receptor pair names for further use
ligand_receptor_names = []
for lr in keeped_pairs.keys():
    ligand = lr[0]
    receptor = lr[1]
    ligand_receptor_names.append(ligand + '-' + receptor)

# Ligand diffusion functions 

Since ligand is a protein that is secreted by the cell, it may float in the cyto fluid and travel to adjacent area. On the contrary, receptor, is usually fixed on the cell membrane. Thus, we "expand" the ligand in a particualr spot to the neighboring 6 spots. That is, if spot A has ligand L gene expression level of 2, then 2 will be add to the L expression level of all neghboring spots of A. We call it "Ligand Diffusion". After that, we take ligand - receptor product.

In [ ]:
def calculate_ligand_diffusion(adata, ligand):
    ligand_positive_idx = find_ligand_positive_idx(adata, ligand)
    ligand_neighbors = find_neighbours(ligand_positive_idx, adata.obsm['spatial'])
    ligand_diffuse_name = apply_ligand_diffusion(adata, ligand_neighbors, ligand)
    return ligand_diffuse_name

In [ ]:
def find_ligand_positive_idx(adata, ligand):
    ligand_positive = []
    for i in range(adata.shape[0]):
        if adata[i, ligand].X.toarray()[0][0] > 0:
            ligand_positive.append((i, adata.obsm['spatial'][i]))
    return ligand_positive

In [ ]:
def find_neighbours(ligand_positive, coordinates):
    coordinates_dict = dict()

    for k, corr1 in ligand_positive:
        coordinates_dict[k] = []

        #calculate the distance of this corr to other corrs in the invasive_edge
        corr_dist_to_others = []
        for corr2 in coordinates:
            corr_dist_to_others.append(get_distance(corr1, corr2))


        #nearest 18 spots (2 laryer spots), exclude itself
        nearest_spots = sorted(range(len(corr_dist_to_others)), key=lambda i: corr_dist_to_others[i])[:7]

        for neighbor in nearest_spots:
            coordinates_dict[k].append(neighbor)
        
    return coordinates_dict

In [ ]:
def get_distance(corr1, corr2):
    return np.sum((corr1 - corr2)**2)

In [ ]:
def apply_ligand_diffusion(adata, coordinates_dict, ligand):
    ligand_expression = []

    for corr, neighbours in coordinates_dict.items():
        #if this ligand is expressed in this spot
        ligand_level = adata[corr, ligand].X.toarray()[0][0]
        for neighbor in neighbours:
            ligand_expression.append((neighbor, ligand_level))
    

    ligand_diffusion = np.zeros(adata.shape[0])
    for neighbor, expression in ligand_expression:
        ligand_diffusion[neighbor] += expression
        
    ligand_diff_name = ligand + ' Diffusion'    
    adata.obs[ligand_diff_name] = ligand_diffusion
    return ligand_diff_name

# Ligand-receptor Product with Diffusion

In [ ]:
def output_LR_fighure(ith_LR):
    dark_image_idx = [9, 10, 11, 12, 13, 14, 27, 28, 29, 30, 31, 32, 33, 34, 35]
    sc.set_figure_params(fontsize = 50, dpi=10, dpi_save=30)

#     print("Progress: ", ith_LR, '/',len(ligand_receptor_names))
    fig, ax = plt.subplots(4,9, figsize=(120, 50))
    for p in range(4):
        for t in range(9):

            idx = 9 * p + t

            if idx in dark_image_idx:
                alpha_img = 0.6
                alpha = 0.7
            else:
                alpha_img = 0.3
                alpha = 0.9

            adata = adata_l[3 * p + t // 3]
            if t % 3 == 0:    
                this_gene = keeped_pairs[ith_LR][0]
                this_gene = this_gene + ' Diffusion'
                this_gene_exp = adata.obs[this_gene].values
#                 this_gene_exp = adata[:, this_gene].X.toarray().flatten()
                this_gene_exp = [q for q in this_gene_exp if q != 0]
                if len(this_gene_exp) > 0:
                    vmax = np.percentile(this_gene_exp, 90)
                else:
                    vmax = 0
                sc.pl.spatial(adata, img_key="hires", color=this_gene, 
                              alpha = alpha, alpha_img = alpha_img, ax = ax[p, t], 
                              title = 'Ligand: ' + this_gene, vmin = 0, vmax = vmax,
                              color_map = 'Greens', colorbar_loc = 'right', show = False)
            elif t % 3 == 1: 
                this_gene = keeped_pairs[ith_LR][1]
                this_gene_exp = adata[:, this_gene].X.toarray().flatten()
                this_gene_exp = [q for q in this_gene_exp if q != 0]
                if len(this_gene_exp) > 0:
                    vmax = np.percentile(this_gene_exp, 90)
                else:
                    vmax = 0
                sc.pl.spatial(adata, img_key="hires", color=this_gene, 
                              alpha = alpha, alpha_img = alpha_img, ax = ax[p, t], 
                              title = 'Receptor: ' + this_gene, vmin = 0, vmax = vmax,
                              color_map = 'Blues', colorbar_loc = 'right', show = False)
            elif t % 3 == 2: 
                this_lr = adata.obs[ligand_receptor_names[ith_LR]].values
                this_lr = [q for q in this_lr if q != 0]
                if len(this_lr) > 0:
                    vmax = np.percentile(this_lr, 90)
                else:
                    vmax = 0
                sc.pl.spatial(adata, img_key="hires", color=ligand_receptor_names[ith_LR], 
                              alpha = alpha, alpha_img = alpha_img, ax = ax[p, t], 
                              title = 'Product: ' + ligand_receptor_names[ith_LR], vmin = 0, vmax = vmax,
                              color_map = 'Purples', colorbar_loc = 'right', show = False)                

    fig.suptitle(ligand_receptor_names[ith_LR], fontsize = 100)
    plt.savefig('./Ligand-Receptor/plots/Diffusion/' + ligand_receptor_names[ith_LR] + '.pdf')
    plt.close(fig)

In [ ]:
counter = 0
for lr, slides in keeped_pairs.items():
    counter += 1
    print('Progress: %d / %d' %(counter, len(keeped_pairs)))
    ligand = lr[0]
    receptor = lr[1]
    
    for k in slides:
        adata = adata_l[k]
        adata.var_names_make_unique()
        
        ligand_diff_name = calculate_ligand_diffusion(adata, ligand)
        ligand_exp = adata.obs[ligand_diff_name].values
#         ligand_exp = adata[:, ligand].X.toarray().flatten()
        recpt_exp = adata[:, receptor].X.toarray().flatten()
        lr_product = ligand_exp * recpt_exp
        ligand_receptor_name = ligand + '-' + receptor
        adata.obs[ligand_receptor_name] = lr_product
        
        #here we just get data, not plot, without normalization
        np.savetxt("./Ligand-Receptor/slide_lr_diffusion/" + "slide-" + str(k) + "-" + ligand_receptor_name + ".txt", 
                   lr_product.astype(int), fmt = '%i')


We are also curious about what if we normalized each spot

In [ ]:
unnorm_adata_l = []
for i in range(12):
    adata = sc.read_visium(visium_paths[i])
    unnorm_adata_l.append(adata)

In [ ]:
# 加入normalization
for i in range(12):
    adata = norm_adata_l[i]
    sc.pp.normalize_total(adata, inplace=True, target_sum=1e4)

In [ ]:
for p in range(0, len(keeped_pairs)):
    lr = list(keeped_pairs)[p]
    slides = list(keeped_pairs.values())[p]
    print('Progress: %d / %d' %(p, len(keeped_pairs)))
    ligand = lr[0]
    receptor = lr[1]
    
    for k in slides:
        adata = unnorm_adata_l[k]
        adata.var_names_make_unique()
        
#         ligand_diff_name = calculate_ligand_diffusion(adata, ligand)
#         ligand_exp = adata.obs[ligand_diff_name].values
        ligand_exp = adata[:, ligand].X.toarray().flatten()
        recpt_exp = adata[:, receptor].X.toarray().flatten()
        lr_product = ligand_exp * recpt_exp
        ligand_receptor_name = ligand + '-' + receptor
        adata.obs[ligand_receptor_name] = lr_product
        
        
        #here we just get data, not plot, with normalization
        np.savetxt("./Ligand-Receptor/slide_lr_without_diffusion_without_normalization/ligands/" + "slide-" + str(k) + "-" + ligand + ".txt", 
                   ligand_exp.astype(int), fmt = '%i')
        np.savetxt("./Ligand-Receptor/slide_lr_without_diffusion_without_normalization/receptors/" + "slide-" + str(k) + "-" + receptor + ".txt", 
                   recpt_exp.astype(int), fmt = '%i')
        np.savetxt("./Ligand-Receptor/slide_lr_without_diffusion_without_normalization/products/" + "slide-" + str(k) + "-" + ligand_receptor_name + ".txt", 
                   lr_product.astype(int), fmt = '%i')


# Just use top 50 genes to infer GEM level (help Lujia)

Here we help Lujia plot GEM level on her data. GEM is not directly outputed from nHDP but by summing the top 50 genes of each GEM generated by nHDP.

In [ ]:
# GEM_top_genes = pd.read_csv("./Ligand-Receptor/crc_merged_gem_list_filtered_for_paper_03072023.csv", index_col = 0)

In [ ]:
# def check_genes_in_ST(GEM_genes):
#     adata = adata_l[0]
#     adata.var_names_make_unique()
#     ST_genes = list(adata.var_names)
#     common_genes = []
#     for gene in GEM_genes:
#         if str(gene) in ST_genes:
#             common_genes.append(gene)
#     return np.array(common_genes)

In [ ]:
# common_genes = {}
# for i in GEM_top_genes.index:
#     GEM_genes = GEM_top_genes.loc[i, :].values
#     common_genes[i] = check_genes_in_ST(GEM_genes)

In [ ]:
# def plot_globally(gene, title, dpi = 50):
#     fig, ax = plt.subplots(4,3, figsize=(35, 50))
#     sc.set_figure_params(fontsize = 50, dpi=dpi, dpi_save=dpi)
#     dark_image_idx = [3, 4, 9, 10, 11]
#     for p in range(4):
#         for t in range(3):

#             idx = 3*p + t

#             if idx in dark_image_idx:
#                 alpha_img = 0.6
#                 alpha = 0.7
#             else:
#                 alpha_img = 0.3
#                 alpha = 0.8

#             adata = adata_l[idx]
#             this_GEM = adata.obs[gene].values
#             this_GEM = [q for q in this_GEM if q != 0]
#             if len(this_GEM) > 0:
#                 vmax = np.percentile(this_GEM, 99)
#             else:
#                 vmax = 0
#             sc.pl.spatial(adata, img_key="hires", color=gene, 
#                           alpha = alpha, alpha_img = alpha_img, ax = ax[p, t], vmin = 0, vmax = vmax,
#                           color_map = 'Blues', colorbar_loc = 'bottom', title = '', show = False)
#             ax[p, t].set_xlabel('')
#             ax[p, t].set_ylabel('')
            
#     plt.tight_layout()
#     fig.suptitle(title, fontsize = 80, verticalalignment = 'top')
#     plt.savefig('./Ligand-Receptor/Lujia_Verification/' + title + '.pdf')
#     plt.close(fig)

In [ ]:
# def plot_GEM_genes_sum(GEM_name):
#     GEM = common_genes[GEM_name]
#     for i in range(12):
#         adata = adata_l[i]
#         sc.pp.normalize_total(adata, inplace=True, target_sum=1e4)
#         GEM_prof = adata[:, GEM].X.toarray().copy()
#         GEM_gene_sum = np.sum(GEM_prof, axis = 1)
#         adata.obs[GEM_name] = GEM_gene_sum
#     plot_globally(GEM_name, GEM_name)

In [ ]:
# for i in range(155, len(GEM_top_genes.index)):
#     GEM_name = GEM_top_genes.index[i]
#     print("Progress: %d / %d" %(i + 1, len(GEM_top_genes)))
#     plot_GEM_genes_sum(GEM_name)